In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from show_spectra import plot_spectra

In [2]:
DF = pd.read_csv('../data/main_table.csv')
DF = DF.loc[DF.w_median.notnull()]
Q_MEDIAN = np.load('../data/q_median.npy')

is_nonvar = DF.nonvar_match_num == 1
is_var    = DF.var_match_num == 1
is_quasar = DF.quasar_match_num == 1

nonvar_subset = DF[is_nonvar & ~is_var & ~is_quasar]
var_subset    = DF[~is_nonvar & is_var & ~is_quasar]
quasar_subset = DF[~is_nonvar & ~is_var & is_quasar]

In [ ]:
%%capture
# capture the output or else the file becomes too big to commit
for gid in var_subset[var_subset.w_rrms > 0.2]['source_id']:
    plot_spectra(gaia_id=gid, savefig=True, mt=DF, q_med=Q_MEDIAN)

In [ ]:
%%capture
# capture the output or else the file becomes too big to commit
for gid in nonvar_subset[nonvar_subset.w_rrms > 0.2]['source_id']:
    plot_spectra(gaia_id=gid, savefig=True, mt=DF, q_med=Q_MEDIAN)

In [9]:
%%capture
for gid in quasar_subset[quasar_subset.w_rrms > 0.1]['source_id']:
    plot_spectra(gaia_id=gid, savefig=True, mt=DF, q_med=Q_MEDIAN)

In [13]:
%%capture
for gid in quasar_subset[quasar_subset.psfMag_r < 18]['source_id']:
    plot_spectra(gaia_id=gid, savefig=True, mt=DF, q_med=Q_MEDIAN)